In [9]:
import zipfile
import os
import shutil

def process_zips(task_id, zip_a_path, zip_b_path):
    temp_dir_a = os.path.join(task_id, "temp_extract_a")
    temp_dir_b = os.path.join(task_id, "temp_extract_b")
    keys_dict = {}
    js_keys_dict = {}
    
    try:
        with zipfile.ZipFile(zip_a_path, 'r') as zip_a:
            zip_a.extractall(temp_dir_a)
        with zipfile.ZipFile(zip_b_path, 'r') as zip_b:
            zip_b.extractall(temp_dir_b)
       
        folders_b = [d for d in os.listdir(temp_dir_b)
                    if os.path.isdir(os.path.join(temp_dir_b, d))]
       
        for folder in folders_b:
            folder_path_a = os.path.join(temp_dir_a, folder)
            folder_path_b = os.path.join(temp_dir_b, folder)
           
            if os.path.exists(folder_path_a) and os.path.isdir(folder_path_a):
                properties_file_a = os.path.join(folder_path_a, "ApplicationResources.properties")
                properties_file_b = os.path.join(folder_path_b, "ApplicationResources.properties")
               
                if os.path.exists(properties_file_a) and os.path.exists(properties_file_b):
                    print(f"Processing folder: {folder} - ApplicationResources")
                    added_keys, modified_keys = find_added_and_modified_keys(properties_file_a, properties_file_b)
                    if len(added_keys) > 0 or len(modified_keys) > 0:
                        keys_dict[folder] = (added_keys, modified_keys)
                else:
                    if not os.path.exists(properties_file_a):
                        print(f"Warning: ApplicationResources.properties not found in ZIP A folder: {folder}")
                    if not os.path.exists(properties_file_b):
                        print(f"Warning: ApplicationResources.properties not found in ZIP B folder: {folder}")
                
                js_properties_file_a = os.path.join(folder_path_a, "JSApplicationResources.properties")
                js_properties_file_b = os.path.join(folder_path_b, "JSApplicationResources.properties")
                
                if os.path.exists(js_properties_file_a) and os.path.exists(js_properties_file_b):
                    print(f"Processing folder: {folder} - JSApplicationResources")
                    js_added_keys, js_modified_keys = find_added_and_modified_keys(js_properties_file_a, js_properties_file_b)
                    if len(js_added_keys) > 0 or len(js_modified_keys) > 0:
                        js_keys_dict[folder] = (js_added_keys, js_modified_keys)
                else:
                    if os.path.exists(js_properties_file_b) and not os.path.exists(js_properties_file_a):
                        print(f"Warning: JSApplicationResources.properties found only in ZIP B folder: {folder}")
                        # If it's a new JS file in ZIP B, we should add all keys
                        with open(js_properties_file_b, 'r', encoding='utf-8') as f:
                            all_keys = {}
                            for line in f:
                                line = line.strip()
                                if line and not line.startswith('#'):
                                    if '=' in line:
                                        key, value = line.split('=', 1)
                                        all_keys[key.strip()] = value.strip()
                        if all_keys:
                            js_keys_dict[folder] = (all_keys, {})
            else:
                print(f"Folder {folder} exists only in ZIP B, skipping")
               
    finally:
        if os.path.exists(temp_dir_a):
            shutil.rmtree(temp_dir_a)
        if os.path.exists(temp_dir_b):
            shutil.rmtree(temp_dir_b)    
    return keys_dict, js_keys_dict
if __name__ == "__main__":
    print(process_zips("test",r"C:\Users\harish-4072\Downloads\applicationcontrolplus.zip", r"C:\Users\harish-4072\Pictures\vulnerabilitymanagerplus.zip"))

Folder 0_framework exists only in ZIP B, skipping
Processing folder: applicationcontrolplus - ApplicationResources
Processing folder: applicationcontrolplus - JSApplicationResources
Folder bsp_core exists only in ZIP B, skipping
Processing folder: configurations - ApplicationResources
Processing folder: configurations - JSApplicationResources
Folder dc_exclude exists only in ZIP B, skipping
Folder demo exists only in ZIP B, skipping
Processing folder: desktopcentral - ApplicationResources
Processing folder: desktopcentral - JSApplicationResources
Folder devicecontrolplus exists only in ZIP B, skipping
Folder device_mgmt_framework exists only in ZIP B, skipping
Folder device_mgmt_framework_onpremise exists only in ZIP B, skipping
Folder dlp exists only in ZIP B, skipping
Folder edr exists only in ZIP B, skipping
Folder emsalerts exists only in ZIP B, skipping
Folder endpoint_intelligence exists only in ZIP B, skipping
Folder fonts exists only in ZIP B, skipping
Folder idps exists only i

In [2]:
def load_properties_with_duplicates(contents):
    properties = {}
    lines = contents.split('\n')
    
    for line in lines:
        line = line.strip()
        if not line or line.startswith('#') or line.startswith('!'):
            continue
            
        split_index = -1
        for i, char in enumerate(line):
            if char in ['=', ':']:
                split_index = i
                break
                
        if split_index != -1:
            key = line[:split_index].strip()
            value = line[split_index + 1:].strip()
            if key not in properties:
                properties[key] = value
                
    return properties

In [3]:
def find_added_and_modified_keys(ref_file, latest_file):
    

    with open(ref_file, 'r', encoding='utf-8') as f:
        reference_contents = f.read()

    with open(latest_file, 'r', encoding='utf-8') as f:
        latest_contents = f.read()
    
    reference_keys = load_properties_with_duplicates(reference_contents)
    latest_keys = load_properties_with_duplicates(latest_contents)
    added_dict = {}
    modified_dict = {}
    
    for key in latest_keys:
        if key not in reference_keys:
            added_dict[key] = latest_keys[key]
        elif reference_keys[key] != latest_keys[key]:
            modified_dict[key] = latest_keys[key]
    
    return added_dict, modified_dict